In [29]:
import numpy as np
import csv
import os
import math
import random
import copy

Q = 4 # Number of available queues
F = 10 # Number of flows
M = 2 # Number of malicious flows
T = 10 # Cycle length [s]
T_END = 100 # Simulation time [s]
TH = 10 # Threshold for queue size increase [Mb]

L = 50.0 # Link speed [Mbps]
R_A = 2.0 # Rate avg [Mbps]
R_S = 0.5  # Rate stdev [Mbps]
D = 50.0  # DDos rate [Mbps]

# make output file
#dirPath = f'test{MinLink}{MaxLink}_{N}_{F}_{T}'
#os.mkdir(dirPath)
#path = f'test{MinLink}{MaxLink}_{N}_{F}_{T}/test{MinLink}{MaxLink}_{N}_{F}_{T}.cnf.txt'

path1 = f'queueSize_{Q}_{F}.txt'
with open(path1, mode='w') as outfile:
    outfile.write('')

path2 = f'queueMap_{Q}_{F}.txt'
with open(path2, mode='w') as outfile:
    outfile.write('')

path3 = f'numAllocatedFlow_{Q}_{F}.txt'
with open(path3, mode='w') as outfile:
    outfile.write('')

In [30]:
### Iitialization
qSize = np.zeros(Q, dtype=int)
qMem = copy.copy(qSize)
qFlag = np.zeros(Q, dtype=int)
fSize = np.zeros(F, dtype=int)
fFlag = np.zeros(F, dtype=int)

# Random allocation
qMap = np.zeros(F, dtype=int)
for i in range(F):
    qMap[i] = random.randint(1, Q-1)
print(qMap)

[3 3 2 3 2 3 1 3 1 1]


In [31]:
# enque
for f in range(F):
    if qMap[f] >= 0:
        if f < M:
            fSize[f] = D
            qSize[qMap[f]] += D # DDoS
        else:
            fSize[f] = np.random.normal(R_A, R_S)
            qSize[qMap[f]] += fSize[f]
    else:# drop
        qSize[qMap[f]] += 0
    print(qSize)

for t in range(0, T_END, T):   
    # Step 1: Queue size evaluation
    for q in range(Q):
        if qSize[q] > TH:
            if q > 0:
                qFlag[q] = 1
            else:
                qFlag[q] = 0
        else:
            qFlag[q] = 0
    qMem = copy.copy(qSize)
     
    # Count number of flows assigned to each queue
    numAllocatedFlow = np.zeros(Q, dtype=int)
    for f in range(F):
        if qMap[f] != -1:
            numAllocatedFlow[qMap[f]] += 1
    # output to file of number of flows assigned to each queue
    with open(path3, mode='a') as outfile:
        for q in range(Q):
            outfile.write(str(numAllocatedFlow[q]))
            if q == Q-1:
                outfile.write('\n')
            else:
                outfile.write('\t')
        
    # Flow identification
    for f in range(F):
        if qMap[f] == -1:
            fFlag[f] = 2
        elif qFlag[qMap[f]] == 1:
            if numAllocatedFlow[qMap[f]] == 1:
                fFlag[f] = 2 # DDoS identified
            else:
                fFlag[f] = 1
        else:
            fFlag[f] = 0
            
    # Step 2: Queue reallocation
    for f in range(F):
        if fFlag[f] == 2:
            qMap[f] = -1 # Drop
        elif fFlag[f] == 1: # reallocate
            qSize[qMap[f]] -= fSize[f]
            qMap[f] = random.randint(1, Q-1)
            qSize[qMap[f]] += fSize[f]
        else: # escape
            if qMap[f] != 0:
                qSize[qMap[f]] -= fSize[f]
                qMap[f] = 0 
                qSize[qMap[f]] += fSize[f]

        
    # write queueMap file
    with open(path2, mode='a') as outfile:
        for f in range(F):
            outfile.write(str(qMap[f]))
            if f == F-1:
                outfile.write('\n')
            else:
                outfile.write('\t')  
    
    # write queueSize file
    with open(path1, mode='a') as outfile:
        outfile.write(str(t))
        for q in range(Q):
            outfile.write('\t' + str(qSize[q]))
            if q == Q-1:
                outfile.write('\n')


[ 0  0  0 50]
[  0   0   0 100]
[  0   0   2 100]
[  0   0   2 101]
[  0   0   4 101]
[  0   0   4 102]
[  0   1   4 102]
[  0   1   4 104]
[  0   1   4 104]
[  0   3   4 104]
